In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Load the classification model
model = load_model(r"screw_bolt_model.h5", compile=False)
class_names = [line.strip() for line in open(r"labels.txt", "r").readlines()]

# Load the image
image_path = r"C:\Users\Admin\Desktop\images (7).jpeg"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Initialize counts
screw_count = 0
bolt_count = 0

def on_trackbar(val):
    global image, gray, model, class_names, screw_count, bolt_count
    
    # Get the trackbar values
    blur_ksize = cv2.getTrackbarPos('Blur Kernel Size', 'Detected Screws and Bolts')
    threshold_value = cv2.getTrackbarPos('Threshold', 'Detected Screws and Bolts')
    min_contour_area = cv2.getTrackbarPos('Min Contour Area', 'Detected Screws and Bolts')

    # Apply Gaussian blur with the current trackbar value
    if blur_ksize % 2 == 0:  # Ensure kernel size is odd
        blur_ksize += 1
    blurred = cv2.GaussianBlur(gray, (blur_ksize, blur_ksize), 0)
    
    # Apply thresholding with the current trackbar value
    _, binary = cv2.threshold(blurred, threshold_value, 255, cv2.THRESH_BINARY_INV)
    
    # Optional: Apply morphological operations
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Reset counts
    screw_count = 0
    bolt_count = 0
    
    # Create a copy of the original image to draw on
    output_image = image.copy()
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_contour_area:
            # Get the bounding box for the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Extract the region of interest (ROI)
            roi = image[y:y+h, x:x+w]

            # Preprocess the ROI for the model
            roi_resized = cv2.resize(roi, (224, 224))
            roi_array = np.asarray(roi_resized, dtype=np.float32).reshape(1, 224, 224, 3)
            roi_array = (roi_array / 127.5) - 1

            # Predict the class of the ROI
            prediction = model.predict(roi_array)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # Print prediction and confidence score for debugging
            print(f"Class: {class_name}, Confidence Score: {confidence_score}")

            # Count screws and bolts
            if 'screw' in class_name and confidence_score > 0.5:  # Adjust the confidence threshold as needed
                screw_count += 1
                label = "Screw"
                color = (0, 255, 0)
            elif 'bolt' in class_name and confidence_score > 0.5:  # Adjust the confidence threshold as needed
                bolt_count += 1
                label = "Bolt"
                color = (255, 0, 0)
            else:
                label = "Unknown"
                color = (0, 0, 255)

            # Draw the bounding box and label on the output image
            cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show the image with detected and labeled screws/bolts
    output_image = cv2.resize(output_image, (700, 700))
    cv2.imshow("Detected Screws and Bolts", output_image)

    # Print the number of screws and bolts detected
    print(f"Number of screws detected: {screw_count}")
    print(f"Number of bolts detected: {bolt_count}")

# Create a window
cv2.namedWindow('Detected Screws and Bolts')

# Create trackbars for different parameters
cv2.createTrackbar('Blur Kernel Size', 'Detected Screws and Bolts', 5, 21, on_trackbar)  # Kernel size must be odd
cv2.createTrackbar('Threshold', 'Detected Screws and Bolts', 127, 255, on_trackbar)
cv2.createTrackbar('Min Contour Area', 'Detected Screws and Bolts', 100, 1000, on_trackbar)

# Initialize the first call to the on_trackbar function
on_trackbar(0)

# Wait for a key press to close the window
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing import image

classifier = load_model('screw_bolt_model.h5')

test_image = tf.keras.utils.load_img(r"C:\Users\Admin\Desktop\images (13).jpeg", target_size=(224,224))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = classifier.predict(test_image)
class_indices = {'bolt': 0, 'screw': 1}

predicted_class_index = np.argmax(result) # used to get maximum value of the index
predicted_class_label = list(class_indices.keys())[predicted_class_index] # used to get class lable based on predicted index

print("Yes! It's " + predicted_class_label)
print(result)

In [17]:
import cv2
import numpy as np

def extract_features(contour):
    x, y, w, h = cv2.boundingRect(contour)

    aspect_ratio = float(w) / h

    area = cv2.contourArea(contour)
    rect_area = w * h
    extent = float(area) / rect_area if rect_area > 0 else 0

    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area if hull_area > 0 else 0

    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments).flatten()

    return aspect_ratio, extent, solidity, circularity, hu_moments

image_path = r"C:\Users\Admin\Desktop\images (9).jpeg"
image = cv2.imread(image_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (5, 5), 0)

threshold = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

screw_count = 0
bolt_count = 0
min_contour_area = 300

for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_contour_area:
        x, y, w, h = cv2.boundingRect(contour)

        aspect_ratio, extent, solidity, circularity, hu_moments = extract_features(contour)

        if circularity < 0.5 and aspect_ratio > 1.2 and solidity < 0.9:
            screw_count += 1
            label = "Screw"
            color = (0, 255, 0)
        else:
            bolt_count += 1
            label = "Bolt"
            color = (255, 0, 0)

        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

print(f"Number of screws detected: {screw_count}")
print(f"Number of bolts detected: {bolt_count}")

image = cv2.resize(image, (700, 700))
cv2.imshow("Detected Screws and Bolts", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of screws detected: 1
Number of bolts detected: 2


# AI and non-AI techniqe

In [3]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model(r"screw_bolt_model.h5", compile=False)
class_names = [line.strip() for line in open("labels.txt", "r").readlines()]

image_path = r"C:\Users\Admin\Downloads\ScrewAndBolt_20240713-20240731T091427Z-001\ScrewAndBolt_20240713\20240713_194606.jpg"
image_read = cv2.imread(image_path)

gray = cv2.cvtColor(image_read, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (3, 3), 0)

_, threshold = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV)

contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

screw_count = 0
bolt_count = 0
min_contour_area = 300

for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_contour_area:
        x, y, w, h = cv2.boundingRect(contour)

        roi = image_read[y:y+h, x:x+w]

        resize_roi = cv2.resize(roi, (224, 224))
        roi_array = np.asarray(resize_roi, dtype=np.float32).reshape(1, 224, 224, 3)
        array_roi = (roi_array / 127.5) - 1

        prediction = model.predict(array_roi)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        print(f"Class: {class_name}, Confidence Score: {confidence_score}")

        if 'screw' in class_name and confidence_score > 0.5:  # Adjust the confidence threshold as needed
            screw_count += 1
            label = "Screw"
            color = (0, 255, 0)
        elif 'bolt' in class_name and confidence_score > 0.5:  # Adjust the confidence threshold as needed
            bolt_count += 1
            label = "Bolt"
            color = (255, 0, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(image_read, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image_read, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


print(f"Number of screws detected: {screw_count}")
print(f"Number of bolts detected: {bolt_count}")

image = cv2.resize(image_read, (700, 700))
cv2.imshow("Detected Screws and Bolts", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 1s 611ms/step
Class: 1 bolt, Confidence Score: 0.9984970092773438
1/1 [==============================] - 0s 33ms/step
Class: 1 bolt, Confidence Score: 0.9984147548675537
1/1 [==============================] - 0s 30ms/step
Class: 1 bolt, Confidence Score: 0.998408854007721
1/1 [==============================] - 0s 42ms/step
Class: 1 bolt, Confidence Score: 0.9980762004852295
1/1 [==============================] - 0s 42ms/step
Class: 1 bolt, Confidence Score: 0.9984744191169739
1/1 [==============================] - 0s 37ms/step
Class: 1 bolt, Confidence Score: 0.9987234473228455
1/1 [==============================] - 0s 39ms/step
Class: 1 bolt, Confidence Score: 0.9999685287475586
1/1 [==============================] - 0s 42ms/step
Class: 1 bolt, Confidence Score: 0.9990853071212769
1/1 [==============================] - 0s 33ms/step
Class: 1 bolt, Confidence Score: 0.9971121549606323
1/1 [==============================] - 0s 42ms/step
Class: 1 bol